In [1]:
# Installation des librairies nécessaires
!pip install num2words
!spacy download fr_core_news_md
!pip install spacy

     |████████████████████████████████| 102kB 3.3MB/s 
     |████████████████████████████████| 88.6MB 1.2MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338489 sha256=4997b3d0d253f8ccdc9c2317a7a28e42c2c981340141c73e9f92582d59a499ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-xt_280qe/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 500
import re

import tensorflow as tf 
import tensorflow_datasets as tfds

from num2words import num2words
import spacy
import fr_core_news_md
nlp = fr_core_news_md.load()

In [3]:
data = pd.read_csv('data.csv')

In [4]:
data.head()

,TEXT,LYRICS
0,"Lorsque débute la Première Guerre mondiale en 1914, \nles pays européens exercent leur domination sur le monde, \npar le biais de la colonisation.","1914-1918, 1ère guerre mondiale\r\nLes pays européens sont présents partout dans le monde\r\nEt ils exercent leur domination\r\nPar le biais de la colonisation\r"
1,"la Grande-Bretagne (l’empire britannique est le plus vaste) et la France, qui colonisent l’Afrique, l’Asie et l’Océanie. La Russie n’a pas de colonies mais elle profite d’un territoire immense et très peuplé.","La France et l’empire britannique sont en tête de liste\r\nEn Asie, en Océanie et en Afrique\r\nLa Russie ne possède pas de colonie\r\nMais c’est malgré tout un très grand pays\r\n"
2,"Si la fin du XIXe siècle est marquée par la suprématie européenne, elle se caractérise aussi par de multiples tensions et rivalités. L’attentat de Sarajevo le 28 juin 1914 est l’événement déclencheur d’une guerre d’abord européenne, puis mondiale.",Beaucoup de rivalité entre les pays\r\nEt montée du sentiment nationaliste\r\nLes différentes puissances nouent des alliances\r\nEt multiplient leurs dépenses d’armements\r
3,"Dans ce contexte, les principales puissances européennes nouent rapidement des alliances : l’Allemagne et l’Autriche-Hongrie créent avec l’Italie la Triple Alliance, tandis que la France, la Russie et l’Angleterre se rapprochent dans une Triple Entente. En parallèle, chaque État multiplie ses dépenses d’armement. L’Allemagne est le pays qui investit le plus pour la fabrication d’armes.","L’Europe se divise en deux grands camps\nLes Empires Centraux face à la Triple entente\nAllemagne, Autriche-Hongrie et plus tard l’Italie\nFace à la France, Le Royaume-Uni et La Russie"
4,"L’archiduc François Ferdinand était l’héritier du trône d’Autriche-Hongrie, rivale de la Serbie dans la région dite des Balkans. Il est assassiné à Sarajevo (en Bosnie-Herzégovine, annexée depuis 1908 par l’Autriche-Hongrie) le 28 juin 1914 par un nationaliste serbe.","Assassinat à Sarajevo en 1914\r\nDe l’héritier du trône d’Autriche-Hongrie, François Ferdinand\r\nIl se fait assassiner par un serbe nationaliste\r\nL’Autriche-Hongrie déclare la guerre à la Serbie\r"


In [5]:
data.shape

(611, 2)

### Preprocessing

In [6]:
# Transformation des chiffres en mots
def numbers_to_words(string):
  string = string.split()
  string = [ num2words(el, lang='fr') if el.isdigit() else el for el in string ]
  string = ' '.join(string)
  return string

In [7]:
# Suppression des chiffres inutils
def remove_single_number(string): 
    pattern = '[0-9]'
    string = string.split()
    string = [re.sub(pattern, '', i) for i in string] 
    string = ' '.join(string)
    return string

In [8]:
# Suppression des caractères spéciaux
def remove_special_characters(text):
  text = text.lower()
  text = text.replace('\r\n', ' <lb> ') # préparation d'encodage des retours à la ligne
  text = re.sub("[|\^&+\-%*/=!>:]\'", '', text)
  text = re.sub(r'\([^)]*\)', '', text)
  text = text.replace(',', '')
  text = text.replace('.', '')
  text = text.replace('-', ' ')
  text = " ".join(text.split())
  return text

In [9]:
# Fonction de preprocessing générale
def preprocess_string(string):
  string = remove_special_characters(string)
  string = numbers_to_words(string)
  string = remove_single_number(string)
  #string = remove_stop_words(string)
  return string

In [10]:
data['TEXT'] = data['TEXT'].apply(preprocess_string)
data['LYRICS'] = data['LYRICS'].apply(preprocess_string)

In [11]:
data.head()

,TEXT,LYRICS
0,lorsque débute la première guerre mondiale en mille neuf cent quatorze les pays européens exercent leur domination sur le monde par le biais de la colonisation,mille neuf cent quatorze mille neuf cent dix-huit ère guerre mondiale <lb> les pays européens sont présents partout dans le monde <lb> et ils exercent leur domination <lb> par le biais de la colonisation
1,la grande bretagne et la france qui colonisent l’afrique l’asie et l’océanie la russie n’a pas de colonies mais elle profite d’un territoire immense et très peuplé,la france et l’empire britannique sont en tête de liste <lb> en asie en océanie et en afrique <lb> la russie ne possède pas de colonie <lb> mais c’est malgré tout un très grand pays <lb>
2,si la fin du xixe siècle est marquée par la suprématie européenne elle se caractérise aussi par de multiples tensions et rivalités l’attentat de sarajevo le vingt-huit juin mille neuf cent quatorze est l’événement déclencheur d’une guerre d’abord européenne puis mondiale,beaucoup de rivalité entre les pays <lb> et montée du sentiment nationaliste <lb> les différentes puissances nouent des alliances <lb> et multiplient leurs dépenses d’armements
3,dans ce contexte les principales puissances européennes nouent rapidement des alliances : l’allemagne et l’autriche hongrie créent avec l’italie la triple alliance tandis que la france la russie et l’angleterre se rapprochent dans une triple entente en parallèle chaque état multiplie ses dépenses d’armement l’allemagne est le pays qui investit le plus pour la fabrication d’armes,l’europe se divise en deux grands camps les empires centraux face à la triple entente allemagne autriche hongrie et plus tard l’italie face à la france le royaume uni et la russie
4,l’archiduc françois ferdinand était l’héritier du trône d’autriche hongrie rivale de la serbie dans la région dite des balkans il est assassiné à sarajevo le vingt-huit juin mille neuf cent quatorze par un nationaliste serbe,assassinat à sarajevo en mille neuf cent quatorze <lb> de l’héritier du trône d’autriche hongrie françois ferdinand <lb> il se fait assassiner par un serbe nationaliste <lb> l’autriche hongrie déclare la guerre à la serbie


In [12]:
# Ajout des balises <start> et <end> sur la target avant l'encodage

def begin_end_sentence(sentence):
  sentence = "<start> "+ sentence + " <end>"
  return sentence

data['LYRICS'] = data['LYRICS'].apply(lambda x: begin_end_sentence(x))


In [13]:
# Ajout des balises spéciales dans le corpus

from spacy.symbols import ORTH

start_case = [{ORTH:"<start>"}]
end_case = [{ORTH: "<end>"}]
line_break = [{ORTH: "<lb>"}]

nlp.tokenizer.add_special_case("<start>", start_case)
nlp.tokenizer.add_special_case("<end>", end_case)
nlp.tokenizer.add_special_case("<lb>", line_break)

In [14]:
data.head()

,TEXT,LYRICS
0,lorsque débute la première guerre mondiale en mille neuf cent quatorze les pays européens exercent leur domination sur le monde par le biais de la colonisation,<start> mille neuf cent quatorze mille neuf cent dix-huit ère guerre mondiale <lb> les pays européens sont présents partout dans le monde <lb> et ils exercent leur domination <lb> par le biais de la colonisation <end>
1,la grande bretagne et la france qui colonisent l’afrique l’asie et l’océanie la russie n’a pas de colonies mais elle profite d’un territoire immense et très peuplé,<start> la france et l’empire britannique sont en tête de liste <lb> en asie en océanie et en afrique <lb> la russie ne possède pas de colonie <lb> mais c’est malgré tout un très grand pays <lb> <end>
2,si la fin du xixe siècle est marquée par la suprématie européenne elle se caractérise aussi par de multiples tensions et rivalités l’attentat de sarajevo le vingt-huit juin mille neuf cent quatorze est l’événement déclencheur d’une guerre d’abord européenne puis mondiale,<start> beaucoup de rivalité entre les pays <lb> et montée du sentiment nationaliste <lb> les différentes puissances nouent des alliances <lb> et multiplient leurs dépenses d’armements <end>
3,dans ce contexte les principales puissances européennes nouent rapidement des alliances : l’allemagne et l’autriche hongrie créent avec l’italie la triple alliance tandis que la france la russie et l’angleterre se rapprochent dans une triple entente en parallèle chaque état multiplie ses dépenses d’armement l’allemagne est le pays qui investit le plus pour la fabrication d’armes,<start> l’europe se divise en deux grands camps les empires centraux face à la triple entente allemagne autriche hongrie et plus tard l’italie face à la france le royaume uni et la russie <end>
4,l’archiduc françois ferdinand était l’héritier du trône d’autriche hongrie rivale de la serbie dans la région dite des balkans il est assassiné à sarajevo le vingt-huit juin mille neuf cent quatorze par un nationaliste serbe,<start> assassinat à sarajevo en mille neuf cent quatorze <lb> de l’héritier du trône d’autriche hongrie françois ferdinand <lb> il se fait assassiner par un serbe nationaliste <lb> l’autriche hongrie déclare la guerre à la serbie <end>


In [15]:
# Création du corpus

array_text = data['TEXT'].values 
array_lyrics = data['LYRICS'].values
array_lyrics = ' '.join(array_lyrics)
array_text = ' '.join(array_text)
all_text = array_lyrics + array_text 

In [16]:
nlp.max_length = len(all_text)

doc = nlp(all_text)

In [17]:
# Création du vocabulaire

tokens = [token.text for token in doc]
vocabulary_set = set(tokens)
vocab_size = len(vocabulary_set)
print(vocab_size)

4407


In [18]:
# Création du dictionnaire d'encodage
 
all_tokens = {}
for i,token in enumerate(vocabulary_set):
  all_tokens[token] = i+1 

In [19]:
# Tokenization

data['TEXT_TOKENS'] = data['TEXT'].map(lambda x : nlp.tokenizer(x))
data['LYRICS_TOKENS'] = data['LYRICS'].map(lambda x : nlp.tokenizer(x))

In [20]:
# Création des indices

def tokens_to_index(tokens):
  indices = []
  for token in tokens:
    indices.append(all_tokens[token.text])
  
  return indices

In [21]:
# Encodage

data['TEXT_ENCODED'] = data['TEXT_TOKENS'].map(lambda x: tokens_to_index(x))
data['LYRICS_ENCODED'] = data['LYRICS_TOKENS'].map(lambda x: tokens_to_index(x))


In [22]:
data.head()

,TEXT,LYRICS,TEXT_TOKENS,LYRICS_TOKENS,TEXT_ENCODED,LYRICS_ENCODED
0,lorsque débute la première guerre mondiale en mille neuf cent quatorze les pays européens exercent leur domination sur le monde par le biais de la colonisation,<start> mille neuf cent quatorze mille neuf cent dix-huit ère guerre mondiale <lb> les pays européens sont présents partout dans le monde <lb> et ils exercent leur domination <lb> par le biais de la colonisation <end>,"(lorsque, débute, la, première, guerre, mondiale, en, mille, neuf, cent, quatorze, les, pays, européens, exercent, leur, domination, sur, le, monde, par, le, biais, de, la, colonisation)","(<start>, mille, neuf, cent, quatorze, mille, neuf, cent, dix-huit, ère, guerre, mondiale, <lb>, les, pays, européens, sont, présents, partout, dans, le, monde, <lb>, et, ils, exercent, leur, domination, <lb>, par, le, biais, de, la, colonisation, <end>)","[2126, 3188, 1807, 2874, 4058, 46, 1617, 462, 2667, 3511, 1172, 2473, 2925, 1933, 1359, 2986, 1709, 3561, 892, 2974, 3353, 892, 597, 3940, 1807, 4206]","[1782, 462, 2667, 3511, 1172, 462, 2667, 3511, 2372, 1553, 4058, 46, 787, 2473, 2925, 1933, 2261, 2416, 4316, 1780, 892, 2974, 787, 3389, 3294, 1359, 2986, 1709, 787, 3353, 892, 597, 3940, 1807, 4206, 2872]"
1,la grande bretagne et la france qui colonisent l’afrique l’asie et l’océanie la russie n’a pas de colonies mais elle profite d’un territoire immense et très peuplé,<start> la france et l’empire britannique sont en tête de liste <lb> en asie en océanie et en afrique <lb> la russie ne possède pas de colonie <lb> mais c’est malgré tout un très grand pays <lb> <end>,"(la, grande, bretagne, et, la, france, qui, colonisent, l’, afrique, l’, asie, et, l’, océanie, la, russie, n’, a, pas, de, colonies, mais, elle, profite, d’, un, territoire, immense, et, très, peuplé)","(<start>, la, france, et, l’, empire, britannique, sont, en, tête, de, liste, <lb>, en, asie, en, océanie, et, en, afrique, <lb>, la, russie, ne, possède, pas, de, colonie, <lb>, mais, c’, est, malgré, tout, un, très, grand, pays, <lb>, <end>)","[1807, 1406, 2744, 3389, 1807, 3780, 2060, 4213, 3581, 567, 3581, 1576, 3389, 3581, 1011, 1807, 2366, 582, 3216, 1217, 3940, 3595, 2773, 870, 3766, 3734, 4356, 1505, 1021, 3389, 3462, 815]","[1782, 1807, 3780, 3389, 3581, 2306, 4073, 2261, 1617, 1861, 3940, 655, 787, 1617, 1576, 1617, 1011, 3389, 1617, 567, 787, 1807, 2366, 2769, 2421, 1217, 3940, 3500, 787, 2773, 82, 43, 3273, 4397, 4356, 3462, 428, 2925, 787, 2872]"
2,si la fin du xixe siècle est marquée par la suprématie européenne elle se caractérise aussi par de multiples tensions et rivalités l’attentat de sarajevo le vingt-huit juin mille neuf cent quatorze est l’événement déclencheur d’une guerre d’abord européenne puis mondiale,<start> beaucoup de rivalité entre les pays <lb> et montée du sentiment nationaliste <lb> les différentes puissances nouent des alliances <lb> et multiplient leurs dépenses d’armements <end>,"(si, la, fin, du, xixe, siècle, est, marquée, par, la, suprématie, européenne, elle, se, caractérise, aussi, par, de, multiples, tensions, et, rivalités, l’, attentat, de, sarajevo, le, vingt-huit, juin, mille, neuf, cent, quatorze, est, l’, événement, déclencheur, d’, une, guerre, d’, abord, européenne, puis, mondiale)","(<start>, beaucoup, de, rivalité, entre, les, pays, <lb>, et, montée, du, sentiment, nationaliste, <lb>, les, différentes, puissances, nouent, des, alliances, <lb>, et, multiplient, leurs, dépenses, d’, armements, <end>)","[1616, 1807, 2341, 822, 1644, 3447, 43, 404, 3353, 1807, 561, 2073, 870, 1960, 3194, 3183, 3353, 3940, 2690, 827, 3389, 4312, 3581, 4237, 3940, 3618, 892, 1898, 4366, 462, 2667, 3511, 1172, 43, 3581, 3714, 1516, 3734, 2944, 4058, 3734, 4319, 2073, 3882, 46]","[1782, 1495, 3940, 1583, 194, 2473, 2925, 787, 3389, 379, 822, 3435, 794, 787, 2473, 1218, 4033, 1364, 3792, 3840, 787, 3389, 203, 1324, 1086, 3734, 2468, 2872]"
3,dans ce contexte les principales puissances européennes nouent rapid

In [23]:
def max_len(lines):
  return max(len(line) for line in lines)

In [24]:
text_max_len = max_len(data['TEXT_ENCODED'].to_list())
lyrics_max_len = max_len(data['LYRICS_ENCODED'].to_list())

In [25]:
# Padding et création du dataset tensorflow

padded_text_indices = tf.keras.preprocessing.sequence.pad_sequences(data["TEXT_ENCODED"], maxlen=text_max_len, padding="post")
padded_lyrics_indices = tf.keras.preprocessing.sequence.pad_sequences(data["LYRICS_ENCODED"], maxlen=lyrics_max_len, padding="post")

tf_ds = tf.data.Dataset.from_tensor_slices((padded_text_indices, padded_lyrics_indices))

In [26]:
# Création du train_test et batch

BATCH_SIZE = 16
TAKE_SIZE = int(0.9*len(data)/BATCH_SIZE)
BUFFER_SIZE = TAKE_SIZE * BATCH_SIZE
steps_per_epoch = TAKE_SIZE
embedding_dim = 128
units = 512

tf_ds = tf_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

train_data = tf_ds.take(TAKE_SIZE)
test_data = tf_ds.skip(TAKE_SIZE)

In [27]:
input_text, output_text = next(iter(train_data))
print(input_text.numpy().shape)
print(output_text.numpy().shape)

(16, 84)
(16, 48)


### MODELLING

In [28]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [29]:
encoder = Encoder(vocab_size +1, embedding_dim, units, BATCH_SIZE)

# Echantillon d'output
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(input_text, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (16, 84, 512)
Encoder Hidden state shape: (batch size, units) (16, 512)


In [30]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # Calcul du score "d'attention"
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # On obtient 1 sur le dernier axe car on applique le score à self.V
    # La shape du tenseur avant que l'on applique self.V est (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [31]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (16, 512)
Attention weights shape: (batch_size, sequence_length, 1) (16, 84, 1)


In [32]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # Utilisé pour attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape après embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape après concaténation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # Passage du vecteur concaténé à la couche GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [33]:
decoder = Decoder(vocab_size + 1, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (16, 4408)


In [34]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [35]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [36]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([all_tokens["<start>"]] * BATCH_SIZE, 1)

    
    for t in range(1, targ.shape[1]):
      
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [37]:
import time

EPOCHS = 300
steps_per_epoch = TAKE_SIZE

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(train_data.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 10 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  
  # Enregistrement (checkpoint) du modèle toutes les 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 5.2663
Epoch 1 Batch 10 Loss 4.4480
Epoch 1 Batch 20 Loss 4.7850
Epoch 1 Batch 30 Loss 4.2706
Epoch 1 Loss 4.7206
Time taken for 1 epoch 49.808661460876465 sec

Epoch 2 Batch 0 Loss 4.0876
Epoch 2 Batch 10 Loss 4.1465
Epoch 2 Batch 20 Loss 4.2439
Epoch 2 Batch 30 Loss 4.3527
Epoch 2 Loss 4.2961
Time taken for 1 epoch 4.338777542114258 sec

Epoch 3 Batch 0 Loss 4.1003
Epoch 3 Batch 10 Loss 3.9497
Epoch 3 Batch 20 Loss 4.1334
Epoch 3 Batch 30 Loss 4.0341
Epoch 3 Loss 4.1859
Time taken for 1 epoch 4.143188953399658 sec

Epoch 4 Batch 0 Loss 4.1805
Epoch 4 Batch 10 Loss 4.2261
Epoch 4 Batch 20 Loss 4.0697
Epoch 4 Batch 30 Loss 4.4413
Epoch 4 Loss 4.1081
Time taken for 1 epoch 4.404816627502441 sec

Epoch 5 Batch 0 Loss 3.9075
Epoch 5 Batch 10 Loss 4.0593
Epoch 5 Batch 20 Loss 4.2743
Epoch 5 Batch 30 Loss 3.9038
Epoch 5 Loss 4.0805
Time taken for 1 epoch 4.221145391464233 sec

Epoch 6 Batch 0 Loss 3.8302
Epoch 6 Batch 10 Loss 4.1971
Epoch 6 Batch 20 Loss 3.7577
Epoch 6 

In [38]:
for example, label in test_data.take(3):
# initialisation sur un exemple du test
  hidden = [tf.zeros((1, units))]
  input_t = example[0]
  output_label = label[0]
  enc_out, enc_hidden = encoder(tf.expand_dims(input_t, axis=0), hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([all_tokens["<start>"]], 0)

  result = ""


# Model lyrics
  for t in range(padded_text_indices.shape[-1]):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                          dec_hidden,
                                                          enc_out)


    predicted_id = tf.argmax(predictions[0]).numpy()

    corresponding_word = [word for word, id in all_tokens.items() if id==predicted_id]
    result += corresponding_word[0] + " "

    if corresponding_word[0] == '<end>':
      break

    dec_input = tf.expand_dims([predicted_id], 0)

# Text sentence
  input_sentence = ""
  for token_id in input_t:
    if token_id==0:
      break
    
    corresponding_word = [word for word, id in all_tokens.items() if id==token_id]
    input_sentence += corresponding_word[0] + " "
    if corresponding_word[0] == "<end>":
      break

# True lyrics
  true_translation = ""

  for token_id in output_label:
    if token_id==0:
      break
    corresponding_word = [word for word, id in all_tokens.items() if id==token_id]
    true_translation += corresponding_word[0] + " "
    if corresponding_word[0] == "<end>":
      break 


print("text sentence: {}".format(input_sentence))
print("True lyrics: {}".format(true_translation))
print("Model lyrics: {}".format(result))

text sentence: l' air inspiré entre par le nez descend par la trachée <lb> les bronches les bronchioles jusqu' aux alvéoles pulmonaires <lb> cet air contient tous les gaz présents dans l' atmosphère 
True lyrics: <start> l' air qui est inspiré entre par le nez ensuite il descend par la trachée puis par les bronches les bronchioles jusqu' aux alvéoles pulmonaires cet air contient tous les gaz présents dans l' atmosphère <end> 
Model lyrics: l' air qui est inspiré entre par le nez ensuite il descend par la trachée puis par les bronches les bronchioles jusqu' aux alvéoles pulmonaires cet air contient tous les gaz présents dans l' atmosphère <end> 
